# Data Loading

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Initialize the accelerator
accelerator = Accelerator()

# read data
train_raw = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

# Package Importing

In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoModel, AutoTokenizer, AutoConfig, RobertaTokenizer, RobertaModel
import torch
import torch.nn as nn
from torch.utils.data import Sampler, Dataset, DataLoader
from accelerate import Accelerator
from tqdm import tqdm
import random
import copy
import os
import multiprocessing
from sklearn.model_selection import StratifiedKFold
import string
import re
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from collections import OrderedDict

 # Preprocessing

In [ ]:
# remove duplicates
train_no_duplicates = train_raw.drop_duplicates('text')
# remove contradictory
duplicates_df = train_raw[train_raw.text.duplicated(keep=False)].sort_values('text')
contradictory_tweets = set()
for tweet in list(duplicates_df.text):
    if len(set(duplicates_df[duplicates_df['text'] == tweet].target)) > 1:
        contradictory_tweets.add(tweet)

contradictory_tweets = list(contradictory_tweets)

filtered_df = train_no_duplicates[~train_no_duplicates['text'].isin(contradictory_tweets)]

# Text cleaning function
def clean_text(text):
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    # Remove user @ references and '#' from text
    text = re.sub(r'\@\w+|\#','', text)
    # Remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Convert to lowercase to maintain consistency
    text = text.lower()
    return text

# Apply the text cleaning function
filtered_df['clean_text'] = filtered_df['text'].apply(clean_text)
test['clean_text'] = test['text'].apply(clean_text)

def combine_columns(row):
    values = [f"{col}: {str(row[col])}" for col in row.index[1:-2] if pd.notnull(row[col])]
    return ' '.join(values)

# Combine the three columns into a single column
filtered_df['combined'] = filtered_df.apply(combine_columns, axis=1)
filtered_df['final_text'] =  filtered_df['clean_text']+' '+ filtered_df['combined']

#test set
filtered_test = test.copy()
filtered_test['combined'] = filtered_test.apply(combine_columns, axis=1)
filtered_test['final_text'] =  filtered_test['clean_text']+' '+ filtered_test['combined']

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


18


/tmp/ipykernel_32/2659865553.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['clean_text'] = filtered_df['text'].apply(clean_text)
/tmp/ipykernel_32/2659865553.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['combined'] = filtered_df.apply(combine_columns, axis=1)
/tmp/ipykernel_32/2659865553.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

# BERT Model

In [ ]:
class MyDataset(Dataset):
    def __init__(self, dataframe, text_column, tokenizer, target='target', max_length = 256):
        self.data = []

        for index, row in tqdm(dataframe.iterrows(), total=len(dataframe), ncols=70):
            text = row[text_column]
            tokenized = tokenizer(text, padding="max_length", truncation=True, max_length=256, return_tensors='pt')
            self.data.append(((tokenized['input_ids'][0].to(device), tokenized['attention_mask'][0].to(device)), torch.tensor(row[target]).to(device)))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

    def train_valid_split(self, train_fraction=.8, shuffle=True):
        num_train_examples = int(len(self) * train_fraction)
        train_dataset = copy.deepcopy(self)

        if shuffle:
            random.shuffle(train_dataset.data)

        valid_dataset = copy.deepcopy(train_dataset)
        train_dataset.data = train_dataset.data[:num_train_examples]
        valid_dataset.data = valid_dataset.data[num_train_examples:]

        return train_dataset, valid_dataset

In [ ]:
class MyDatasetTest(Dataset):
    def __init__(self, dataframe, text_column, tokenizer, max_length = 256):
        self.data = []

        for index, row in tqdm(dataframe.iterrows(), total=len(dataframe), ncols=70):
            text = row[text_column]
            tokenized = tokenizer(text, padding="max_length", truncation=True, max_length=256, return_tensors='pt')
            self.data.append((torch.tensor(row['id']).to(device) ,(tokenized['input_ids'][0].to(device), tokenized['attention_mask'][0].to(device))))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
class FinetuneClassifier(nn.Module):
    def __init__(self, model=model_checkpoint, classes=2, head_dropout=0.2):
        super().__init__()

        self.model = AutoModel.from_pretrained(model)
        hidden_size = self.model.config.hidden_size

        self.project = torch.nn.Sequential(
            torch.nn.Dropout(head_dropout),
            torch.nn.Linear(hidden_size, hidden_size),
            torch.nn.Dropout(head_dropout),
            torch.nn.Linear(hidden_size, classes) # projection
        )

    def forward(self, input_ids, attention_mask=None):
        res = self.model.forward(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        res = res[0]
        res = res[:,0,:] # encoding for <s> token
        res = self.project(res)
        return res

    def parameters_num(self):
        return sum(p.numel() for p in self.parameters())

In [ ]:
def train(model,
          train_dataloader,
          valid_dataloader,
          steps,
          optimizer,
          accelerator,
          blind_steps=None,
          loss_fn=torch.nn.BCELoss(),
          main_metric=('f1', f1_score),
          additional_metrics=[],
          filepath='model_best_BERT.pt',
          load_best=True,
          scheduler=None,
          losses_dict=None):

    if blind_steps == None:
        blind_steps = len(train_dataloader) // 4

    def evaluate():  # the first score returned is the main
        model.eval()

        y_trues = []
        y_hats = []

        loss = 0
        k = 0

        with torch.no_grad():
            for batch in valid_dataloader:

                (ids, mask), y_true = batch
                ids, mask = accelerator.prepare(ids), accelerator.prepare(mask)
                y_true = accelerator.prepare(y_true)
                hots = torch.nn.functional.one_hot(y_true, 2).to(dtype=torch.float)
                y_hat = torch.softmax(model.forward(input_ids=ids, attention_mask=mask),dim=-1)

                loss += float(loss_fn(y_hat, hots))
                k += 1

                for i in range(y_true.shape[0]):
                    y_trues.append(int(y_true[i]))
                    y_hats.append(1 if y_hat[i][0] < y_hat[i][1] else 0)

        scores = [(main_metric[0], main_metric[1](y_trues, y_hats))]

        for metric in additional_metrics:
            scores.append((metric[0], metric[1](y_trues, y_hats)))

        model.train()
        return scores + [('valid_loss', loss/k)]


    def render_scores(scores, step, best=None):
        print('{:05d} steps'.format(step), end=' ')

        for score in scores:
            print("| {}: {:.3f}".format(*score), end=' ')

        if best != None:
            print('| best_score: {:.3f}'.format(best))


    # initial scores
    scores = evaluate()
    render_scores(scores, 0)
    best_score = scores[0][1]
    torch.save(accelerator.unwrap_model(model).state_dict(), filepath)

    # logs
    if losses_dict != None:
        losses_dict['train_loss'] = []
        losses_dict['valid_loss'] = []
        losses_dict[main_metric[0]] = []

    epoch_loss = 0
    k = 0

    train_iter = iter(train_dataloader)
    model.train()

    for step in tqdm(range(steps)):

        # retrieving a batch
        try:
            batch = next(train_iter)
        except:
            train_iter = iter(train_dataloader)
            batch = next(train_iter)

        (ids, mask), y_true = batch
        ids, mask = accelerator.prepare(ids), accelerator.prepare(mask)
        y_true = accelerator.prepare(y_true)

        # prediction
        y_hat = torch.softmax(model.forward(input_ids=ids, attention_mask=mask),dim=-1)
        hots = torch.nn.functional.one_hot(y_true, 2).to(dtype=torch.float)
        loss = loss_fn(y_hat, hots)

        # backprop
        optimizer.zero_grad()
        accelerator.backward(loss)
        optimizer.step()

        if scheduler != None:
            scheduler.step()

        epoch_loss += float(loss)
        k += 1

        # evaluation
        if (step + 1) % blind_steps == 0:
            scores = evaluate() + [('train_loss', epoch_loss/k)]

            if losses_dict != None:
                losses_dict['valid_loss'].append(float(scores[-2][1]))
                losses_dict['train_loss'].append(float(scores[-1][1]))
                losses_dict[main_metric[0]].append(float(scores[0][1]))

            if scores[0][1] > best_score:
                best_score = scores[0][1]
                torch.save(accelerator.unwrap_model(model).state_dict(), filepath)

            render_scores(scores, step + 1, best=best_score)
            epoch_loss = 0
            k = 0

    if load_best:
        state_dict = torch.load(filepath)

        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            if "module." not in k:
                name = 'module.' + k
                new_state_dict[name] = v

        model.load_state_dict(new_state_dict)

In [ ]:
device = accelerator.device
model_checkpoint = "bert-large-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Create an instance of your custom Dataset class
dataset = MyDataset(filtered_df, 'final_text', tokenizer)
train_dataset, valid_dataset = dataset.train_valid_split()

dataset_test = MyDatasetTest(filtered_test, 'final_text', tokenizer)

# Create a DataLoader
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

model = FinetuneClassifier(head_dropout=.1)
model = nn.DataParallel(model)
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, total_iters=500)
logs_dict = {}

In [ ]:
train(
  model,
  train_dataloader,
  valid_dataloader,
  2000,
  optimizer,
  accelerator,
  blind_steps=100,
  additional_metrics=[('precision', precision_score), ('recall', recall_score),('accuracy', accuracy_score)],
  losses_dict=logs_dict,
  scheduler=scheduler
)

100%|███████████████████████████| 3263/3263 [00:02<00:00, 1165.81it/s]


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


00000 steps | f1: 0.281 | precision: 0.398 | recall: 0.217 | accuracy: 0.525 | valid_loss: 0.691 

100%|██████████| 100/100 [03:19<00:00,  1.99s/it]

00100 steps | f1: 0.722 | precision: 0.784 | recall: 0.669 | accuracy: 0.780 | valid_loss: 0.489 | train_loss: 0.606 | best_score: 0.722


# RoBERTa Model

In [ ]:
class MyDataset(Dataset):
    def __init__(self, dataframe, text_column, tokenizer, target='target', max_length = 256):
        self.data = []

        for index, row in tqdm(dataframe.iterrows(), total=len(dataframe), ncols=70):
            text = row[text_column]
            tokenized = tokenizer(text, padding="max_length", truncation=True, max_length=256, return_tensors='pt')
            self.data.append(((tokenized['input_ids'][0].to(device), tokenized['attention_mask'][0].to(device)), torch.tensor(row[target]).to(device)))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

    def train_valid_split(self, train_fraction=.8, shuffle=True):
        num_train_examples = int(len(self) * train_fraction)
        train_dataset = copy.deepcopy(self)

        if shuffle:
            random.shuffle(train_dataset.data)

        valid_dataset = copy.deepcopy(train_dataset)
        train_dataset.data = train_dataset.data[:num_train_examples]
        valid_dataset.data = valid_dataset.data[num_train_examples:]

        return train_dataset, valid_dataset

NameError: name 'Dataset' is not defined

In [ ]:
class MyDatasetTest(Dataset):
    def __init__(self, dataframe, text_column, tokenizer, max_length = 256):
        self.data = []

        for index, row in tqdm(dataframe.iterrows(), total=len(dataframe), ncols=70):
            text = row[text_column]
            tokenized = tokenizer(text, padding="max_length", truncation=True, max_length=256, return_tensors='pt')
            self.data.append((torch.tensor(row['id']).to(device) ,(tokenized['input_ids'][0].to(device), tokenized['attention_mask'][0].to(device))))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
class FinetuneClassifier(nn.Module):
    def __init__(self, model=model_checkpoint, classes=2, head_dropout=0.2):
        super().__init__()

        self.model = RobertaModel.from_pretrained(model)
        hidden_size = self.model.config.hidden_size

        self.project = torch.nn.Sequential(
            torch.nn.Dropout(head_dropout),
            torch.nn.Linear(hidden_size, hidden_size),
            torch.nn.Dropout(head_dropout),
            torch.nn.Linear(hidden_size, classes) # projection
        )

    def forward(self, input_ids, attention_mask=None):
        res = self.model.forward(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        res = res[0]
        res = res[:,0,:] # encoding for <s> token
        res = self.project(res)
        return res

    def parameters_num(self):
        return sum(p.numel() for p in self.parameters())

In [ ]:
def train(model,
          train_dataloader,
          valid_dataloader,
          steps,
          optimizer,
          accelerator,
          blind_steps=None,
          loss_fn=torch.nn.BCELoss(),
          main_metric=('f1', f1_score),
          additional_metrics=[],
          filepath='model_best_RoBERTa.pt',
          load_best=True,
          scheduler=None,
          losses_dict=None):

    if blind_steps == None:
        blind_steps = len(train_dataloader) // 4

    def evaluate():  # the first score returned is the main
        model.eval()

        y_trues = []
        y_hats = []

        loss = 0
        k = 0

        with torch.no_grad():
            for batch in valid_dataloader:

                (ids, mask), y_true = batch
                ids, mask = accelerator.prepare(ids), accelerator.prepare(mask)
                y_true = accelerator.prepare(y_true)
                hots = torch.nn.functional.one_hot(y_true, 2).to(dtype=torch.float)
                y_hat = torch.softmax(model.forward(input_ids=ids, attention_mask=mask),dim=-1)

                loss += float(loss_fn(y_hat, hots))
                k += 1

                for i in range(y_true.shape[0]):
                    y_trues.append(int(y_true[i]))
                    y_hats.append(1 if y_hat[i][0] < y_hat[i][1] else 0)

        scores = [(main_metric[0], main_metric[1](y_trues, y_hats))]

        for metric in additional_metrics:
            scores.append((metric[0], metric[1](y_trues, y_hats)))

        model.train()
        return scores + [('valid_loss', loss/k)]


    def render_scores(scores, step, best=None):
        print('{:05d} steps'.format(step), end=' ')

        for score in scores:
            print("| {}: {:.3f}".format(*score), end=' ')

        if best != None:
            print('| best_score: {:.3f}'.format(best))


    # initial scores
    scores = evaluate()
    render_scores(scores, 0)
    best_score = scores[0][1]
    torch.save(accelerator.unwrap_model(model).state_dict(), filepath)

    # logs
    if losses_dict != None:
        losses_dict['train_loss'] = []
        losses_dict['valid_loss'] = []
        losses_dict[main_metric[0]] = []

    epoch_loss = 0
    k = 0

    train_iter = iter(train_dataloader)
    model.train()

    for step in tqdm(range(steps)):

        # retrieving a batch
        try:
            batch = next(train_iter)
        except:
            train_iter = iter(train_dataloader)
            batch = next(train_iter)

        (ids, mask), y_true = batch
        ids, mask = accelerator.prepare(ids), accelerator.prepare(mask)
        y_true = accelerator.prepare(y_true)

        # prediction
        y_hat = torch.softmax(model.forward(input_ids=ids, attention_mask=mask),dim=-1)
        hots = torch.nn.functional.one_hot(y_true, 2).to(dtype=torch.float)
        loss = loss_fn(y_hat, hots)

        # backprop
        optimizer.zero_grad()
        accelerator.backward(loss)
        optimizer.step()

        if scheduler != None:
            scheduler.step()

        epoch_loss += float(loss)
        k += 1

        # evaluation
        if (step + 1) % blind_steps == 0:
            scores = evaluate() + [('train_loss', epoch_loss/k)]

            if losses_dict != None:
                losses_dict['valid_loss'].append(float(scores[-2][1]))
                losses_dict['train_loss'].append(float(scores[-1][1]))
                losses_dict[main_metric[0]].append(float(scores[0][1]))

            if scores[0][1] > best_score:
                best_score = scores[0][1]
                torch.save(accelerator.unwrap_model(model).state_dict(), filepath)

            render_scores(scores, step + 1, best=best_score)
            epoch_loss = 0
            k = 0

    if load_best:
        state_dict = torch.load(filepath)

        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            if "module." not in k:
                name = 'module.' + k
                new_state_dict[name] = v

        model.load_state_dict(new_state_dict)

In [ ]:
device = accelerator.device
model_checkpoint = "roberta-large"

tokenizer = RobertaTokenizer.from_pretrained(model_checkpoint)
model = RobertaModel.from_pretrained(model_checkpoint)

# Create an instance of your custom Dataset class
dataset = MyDataset(filtered_df, 'final_text', tokenizer)
train_dataset, valid_dataset = dataset.train_valid_split()

dataset_test = MyDatasetTest(filtered_test, 'final_text', tokenizer)

# Create a DataLoader
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

model = FinetuneClassifier(head_dropout=.1)
model = nn.DataParallel(model)
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=5e-6, weight_decay=1.5e-3)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, total_iters=500)
logs_dict = {}

In [ ]:
train(
  model,
  train_dataloader,
  valid_dataloader,
  2000,
  optimizer,
  accelerator,
  blind_steps=100,
  additional_metrics=[('precision', precision_score), ('recall', recall_score),('accuracy', accuracy_score)],
  losses_dict=logs_dict,
  scheduler=scheduler
)

# XLNet Model

In [ ]:
class MyDataset(Dataset):
    def __init__(self, dataframe, text_column, tokenizer, target='target', max_length = 256):
        self.data = []

        for index, row in tqdm(dataframe.iterrows(), total=len(dataframe), ncols=70):
            text = row[text_column]
            tokenized = tokenizer(text, padding="max_length", truncation=True, max_length=256, return_tensors='pt')
            self.data.append(((tokenized['input_ids'][0].to(device), tokenized['attention_mask'][0].to(device)), torch.tensor(row[target]).to(device)))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

    def train_valid_split(self, train_fraction=.8, shuffle=True):
        num_train_examples = int(len(self) * train_fraction)
        train_dataset = copy.deepcopy(self)

        if shuffle:
            random.shuffle(train_dataset.data)

        valid_dataset = copy.deepcopy(train_dataset)
        train_dataset.data = train_dataset.data[:num_train_examples]
        valid_dataset.data = valid_dataset.data[num_train_examples:]

        return train_dataset, valid_dataset

In [ ]:
class MyDatasetTest(Dataset):
    def __init__(self, dataframe, text_column, tokenizer, max_length = 256):
        self.data = []

        for index, row in tqdm(dataframe.iterrows(), total=len(dataframe), ncols=70):
            text = row[text_column]
            tokenized = tokenizer(text, padding="max_length", truncation=True, max_length=256, return_tensors='pt')
            self.data.append((torch.tensor(row['id']).to(device) ,(tokenized['input_ids'][0].to(device), tokenized['attention_mask'][0].to(device))))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
class FinetuneClassifier(nn.Module):
    def __init__(self, model=model_checkpoint, classes=2, head_dropout=0.2):
        super().__init__()

        self.model = AutoModel.from_pretrained(model)
        hidden_size = self.model.config.hidden_size

        self.project = torch.nn.Sequential(
            torch.nn.Dropout(head_dropout),
            torch.nn.Linear(hidden_size, hidden_size),
            torch.nn.Dropout(head_dropout),
            torch.nn.Linear(hidden_size, classes) # projection
        )

    def forward(self, input_ids, attention_mask=None):
        res = self.model.forward(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        res = res[0]
        res = res[:,0,:] # encoding for <s> token
        res = self.project(res)
        return res

    def parameters_num(self):
        return sum(p.numel() for p in self.parameters())

In [ ]:
def train(model,
          train_dataloader,
          valid_dataloader,
          steps,
          optimizer,
          accelerator,
          blind_steps=None,
          loss_fn=torch.nn.BCELoss(),
          main_metric=('f1', f1_score),
          additional_metrics=[],
          filepath='model_best_XLNet.pt',
          load_best=True,
          scheduler=None,
          losses_dict=None):

    if blind_steps == None:
        blind_steps = len(train_dataloader) // 4

    def evaluate():  # the first score returned is the main
        model.eval()

        y_trues = []
        y_hats = []

        loss = 0
        k = 0

        with torch.no_grad():
            for batch in valid_dataloader:

                (ids, mask), y_true = batch
                ids, mask = accelerator.prepare(ids), accelerator.prepare(mask)
                y_true = accelerator.prepare(y_true)
                hots = torch.nn.functional.one_hot(y_true, 2).to(dtype=torch.float)
                y_hat = torch.softmax(model.forward(input_ids=ids, attention_mask=mask),dim=-1)

                loss += float(loss_fn(y_hat, hots))
                k += 1

                for i in range(y_true.shape[0]):
                    y_trues.append(int(y_true[i]))
                    y_hats.append(1 if y_hat[i][0] < y_hat[i][1] else 0)

        scores = [(main_metric[0], main_metric[1](y_trues, y_hats))]

        for metric in additional_metrics:
            scores.append((metric[0], metric[1](y_trues, y_hats)))

        model.train()
        return scores + [('valid_loss', loss/k)]


    def render_scores(scores, step, best=None):
        print('{:05d} steps'.format(step), end=' ')

        for score in scores:
            print("| {}: {:.3f}".format(*score), end=' ')

        if best != None:
            print('| best_score: {:.3f}'.format(best))


    # initial scores
    scores = evaluate()
    render_scores(scores, 0)
    best_score = scores[0][1]
    torch.save(accelerator.unwrap_model(model).state_dict(), filepath)

    # logs
    if losses_dict != None:
        losses_dict['train_loss'] = []
        losses_dict['valid_loss'] = []
        losses_dict[main_metric[0]] = []

    epoch_loss = 0
    k = 0

    train_iter = iter(train_dataloader)
    model.train()

    for step in tqdm(range(steps)):

        # retrieving a batch
        try:
            batch = next(train_iter)
        except:
            train_iter = iter(train_dataloader)
            batch = next(train_iter)

        (ids, mask), y_true = batch
        ids, mask = accelerator.prepare(ids), accelerator.prepare(mask)
        y_true = accelerator.prepare(y_true)

        # prediction
        y_hat = torch.softmax(model.forward(input_ids=ids, attention_mask=mask),dim=-1)
        hots = torch.nn.functional.one_hot(y_true, 2).to(dtype=torch.float)
        loss = loss_fn(y_hat, hots)

        # backprop
        optimizer.zero_grad()
        accelerator.backward(loss)
        optimizer.step()

        if scheduler != None:
            scheduler.step()

        epoch_loss += float(loss)
        k += 1

        # evaluation
        if (step + 1) % blind_steps == 0:
            scores = evaluate() + [('train_loss', epoch_loss/k)]

            if losses_dict != None:
                losses_dict['valid_loss'].append(float(scores[-2][1]))
                losses_dict['train_loss'].append(float(scores[-1][1]))
                losses_dict[main_metric[0]].append(float(scores[0][1]))

            if scores[0][1] > best_score:
                best_score = scores[0][1]
                torch.save(accelerator.unwrap_model(model).state_dict(), filepath)

            render_scores(scores, step + 1, best=best_score)
            epoch_loss = 0
            k = 0

    if load_best:
        state_dict = torch.load(filepath)

        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            if "module." not in k:
                name = 'module.' + k
                new_state_dict[name] = v

        model.load_state_dict(new_state_dict)

In [ ]:
device = accelerator.device
model_checkpoint = "xlnet-large-cased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Create an instance of your custom Dataset class
dataset = MyDataset(filtered_df, 'final_text', tokenizer)
train_dataset, valid_dataset = dataset.train_valid_split()

dataset_test = MyDatasetTest(filtered_test, 'final_text', tokenizer)

# Create a DataLoader
batch_size = 12
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

model = FinetuneClassifier(head_dropout=.1)
model = nn.DataParallel(model)
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, total_iters=500)
logs_dict = {}

In [ ]:
train(
  model,
  train_dataloader,
  valid_dataloader,
  2000,
  optimizer,
  accelerator,
  blind_steps=100,
  additional_metrics=[('precision', precision_score), ('recall', recall_score),('accuracy', accuracy_score)],
  losses_dict=logs_dict,
  scheduler=scheduler
)

100%|████████████████████████████| 3263/3263 [00:03<00:00, 967.56it/s]


Some weights of the model checkpoint at xlnet-large-cased were not used when initializing XLNetModel: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


00000 steps | f1: 0.397 | precision: 0.437 | recall: 0.363 | accuracy: 0.524 | valid_loss: 0.872 

100%|██████████| 100/100 [03:58<00:00,  2.38s/it]

00100 steps | f1: 0.312 | precision: 0.608 | recall: 0.210 | accuracy: 0.602 | valid_loss: 0.651 | train_loss: 0.870 | best_score: 0.397


# ELECTRA Model

In [ ]:
class MyDataset(Dataset):
    def __init__(self, dataframe, text_column, tokenizer, target='target', max_length = 256):
        self.data = []

        for index, row in tqdm(dataframe.iterrows(), total=len(dataframe), ncols=70):
            text = row[text_column]
            tokenized = tokenizer(text, padding="max_length", truncation=True, max_length=256, return_tensors='pt')
            self.data.append(((tokenized['input_ids'][0].to(device), tokenized['attention_mask'][0].to(device)), torch.tensor(row[target]).to(device)))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

    def train_valid_split(self, train_fraction=.8, shuffle=True):
        num_train_examples = int(len(self) * train_fraction)
        train_dataset = copy.deepcopy(self)

        if shuffle:
            random.shuffle(train_dataset.data)

        valid_dataset = copy.deepcopy(train_dataset)
        train_dataset.data = train_dataset.data[:num_train_examples]
        valid_dataset.data = valid_dataset.data[num_train_examples:]

        return train_dataset, valid_dataset

In [ ]:
class MyDatasetTest(Dataset):
    def __init__(self, dataframe, text_column, tokenizer, max_length = 256):
        self.data = []

        for index, row in tqdm(dataframe.iterrows(), total=len(dataframe), ncols=70):
            text = row[text_column]
            tokenized = tokenizer(text, padding="max_length", truncation=True, max_length=256, return_tensors='pt')
            self.data.append((torch.tensor(row['id']).to(device) ,(tokenized['input_ids'][0].to(device), tokenized['attention_mask'][0].to(device))))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
class FinetuneClassifier(nn.Module):
    def __init__(self, model=model_checkpoint, classes=2, head_dropout=0.2):
        super().__init__()

        self.model = ElectraModel.from_pretrained(model)
        hidden_size = self.model.config.hidden_size

        self.project = torch.nn.Sequential(
            torch.nn.Dropout(head_dropout),
            torch.nn.Linear(hidden_size, hidden_size),
            torch.nn.Dropout(head_dropout),
            torch.nn.Linear(hidden_size, classes)
        )

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        last_hidden_state = outputs[0]
        cls_output = last_hidden_state[:, 0, :]
        cls_output = self.project(cls_output)
        return cls_output

    def parameters_num(self):
        return sum(p.numel() for p in self.parameters())

In [ ]:
def train(model,
          train_dataloader,
          valid_dataloader,
          steps,
          optimizer,
          accelerator,
          blind_steps=None,
          loss_fn=torch.nn.BCELoss(),
          main_metric=('f1', f1_score),
          additional_metrics=[],
          filepath='model_best_ELECTRA.pt',
          load_best=True,
          scheduler=None,
          losses_dict=None):

    if blind_steps == None:
        blind_steps = len(train_dataloader) // 4

    def evaluate():  # the first score returned is the main
        model.eval()

        y_trues = []
        y_hats = []

        loss = 0
        k = 0

        with torch.no_grad():
            for batch in valid_dataloader:

                (ids, mask), y_true = batch
                ids, mask = accelerator.prepare(ids), accelerator.prepare(mask)
                y_true = accelerator.prepare(y_true)
                hots = torch.nn.functional.one_hot(y_true, 2).to(dtype=torch.float)
                y_hat = torch.softmax(model.forward(input_ids=ids, attention_mask=mask),dim=-1)

                loss += float(loss_fn(y_hat, hots))
                k += 1

                for i in range(y_true.shape[0]):
                    y_trues.append(int(y_true[i]))
                    y_hats.append(1 if y_hat[i][0] < y_hat[i][1] else 0)

        scores = [(main_metric[0], main_metric[1](y_trues, y_hats))]

        for metric in additional_metrics:
            scores.append((metric[0], metric[1](y_trues, y_hats)))

        model.train()
        return scores + [('valid_loss', loss/k)]


    def render_scores(scores, step, best=None):
        print('{:05d} steps'.format(step), end=' ')

        for score in scores:
            print("| {}: {:.3f}".format(*score), end=' ')

        if best != None:
            print('| best_score: {:.3f}'.format(best))


    # initial scores
    scores = evaluate()
    render_scores(scores, 0)
    best_score = scores[0][1]
    torch.save(accelerator.unwrap_model(model).state_dict(), filepath)

    # logs
    if losses_dict != None:
        losses_dict['train_loss'] = []
        losses_dict['valid_loss'] = []
        losses_dict[main_metric[0]] = []

    epoch_loss = 0
    k = 0

    train_iter = iter(train_dataloader)
    model.train()

    for step in tqdm(range(steps)):

        # retrieving a batch
        try:
            batch = next(train_iter)
        except:
            train_iter = iter(train_dataloader)
            batch = next(train_iter)

        (ids, mask), y_true = batch
        ids, mask = accelerator.prepare(ids), accelerator.prepare(mask)
        y_true = accelerator.prepare(y_true)

        # prediction
        y_hat = torch.softmax(model.forward(input_ids=ids, attention_mask=mask),dim=-1)
        hots = torch.nn.functional.one_hot(y_true, 2).to(dtype=torch.float)
        loss = loss_fn(y_hat, hots)

        # backprop
        optimizer.zero_grad()
        accelerator.backward(loss)
        optimizer.step()

        if scheduler != None:
            scheduler.step()

        epoch_loss += float(loss)
        k += 1

        # evaluation
        if (step + 1) % blind_steps == 0:
            scores = evaluate() + [('train_loss', epoch_loss/k)]

            if losses_dict != None:
                losses_dict['valid_loss'].append(float(scores[-2][1]))
                losses_dict['train_loss'].append(float(scores[-1][1]))
                losses_dict[main_metric[0]].append(float(scores[0][1]))

            if scores[0][1] > best_score:
                best_score = scores[0][1]
                torch.save(accelerator.unwrap_model(model).state_dict(), filepath)

            render_scores(scores, step + 1, best=best_score)
            epoch_loss = 0
            k = 0

    if load_best:
        state_dict = torch.load(filepath)

        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            if "module." not in k:
                name = 'module.' + k
                new_state_dict[name] = v

        model.load_state_dict(new_state_dict)

In [ ]:
device = accelerator.device
model_checkpoint = 'google/electra-base-discriminator'

tokenizer = ElectraTokenizer.from_pretrained(model_checkpoint)
model = ElectraForSequenceClassification.from_pretrained(model_checkpoint)

# Create an instance of your custom Dataset class
dataset = MyDataset(filtered_df, 'final_text', tokenizer)
train_dataset, valid_dataset = dataset.train_valid_split()

dataset_test = MyDatasetTest(filtered_test, 'final_text', tokenizer)

# Create a DataLoader
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

model = FinetuneClassifier(head_dropout=.1)
model = nn.DataParallel(model)
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, total_iters=500)
logs_dict = {}

In [ ]:
train(
  model,
  train_dataloader,
  valid_dataloader,
  2000,
  optimizer,
  accelerator,
  blind_steps=100,
  additional_metrics=[('precision', precision_score), ('recall', recall_score),('accuracy', accuracy_score)],
  losses_dict=logs_dict,
  scheduler=scheduler
)

# Predicting

In [ ]:
model_name = 'BERT'

In [ ]:
state_dict = torch.load(f'/kaggle/working/model_best_{model_name}.pt')

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if 'module.' not in k:
        name = 'module.' + k
        new_state_dict[name] = v

model.load_state_dict(new_state_dict)

def evaluate(model, valid_dataloader, metrics=[('f1', f1_score),('precision', precision_score), ('recall', recall_score),('accuracy', accuracy_score)]):
    model.eval()

    y_trues = []
    y_hats = []

    with torch.no_grad():
        for batch in valid_dataloader:

            (ids, mask), y_true = batch
            y_hat = torch.softmax(model.forward(input_ids=ids, attention_mask=mask),dim=-1)

            for i in range(y_true.shape[0]):
                y_trues.append(int(y_true[i]))
                y_hats.append(1 if y_hat[i][0] < y_hat[i][1] else 0)

    scores = []

    for metric in metrics:
        scores.append((metric[0], metric[1](y_trues, y_hats)))

    return scores

scores = evaluate(model, valid_dataloader)
print(scores)

predictions_df = pd.DataFrame()
for i, (ids, mask) in tqdm(dataset_test):
    ids, mask = accelerator.prepare(ids), accelerator.prepare(mask)
    pred = model(input_ids=ids[None], attention_mask=mask[None])[0]
    y_hat = 1 if pred[0] < pred[1] else 0
    r = [int(i), y_hat]
    predictions_df = pd.concat([predictions_df, pd.DataFrame(np.array(r)[None,:], columns=['id', 'target'])])

predictions_df.to_csv('submission.csv', index=False)

# RoBERTa Model with K-Fold

In [ ]:
class MyDataset(Dataset):
    def __init__(self, dataframe, text_column, tokenizer, target='target', max_length = 256):
        self.data = []

        for index, row in tqdm(dataframe.iterrows(), total=len(dataframe), ncols=70):
            text = row[text_column]
            tokenized = tokenizer(text, padding="max_length", truncation=True, max_length=256, return_tensors='pt')
            self.data.append(((tokenized['input_ids'][0].to(device), tokenized['attention_mask'][0].to(device)), torch.tensor(row[target]).to(device)))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
class MyDatasetTest(Dataset):
    def __init__(self, dataframe, text_column, tokenizer, max_length = 256):
        self.data = []

        for index, row in tqdm(dataframe.iterrows(), total=len(dataframe), ncols=70):
            text = row[text_column]
            tokenized = tokenizer(text, padding="max_length", truncation=True, max_length=256, return_tensors='pt')
            self.data.append((torch.tensor(row['id']).to(device) ,(tokenized['input_ids'][0].to(device), tokenized['attention_mask'][0].to(device))))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
class FinetuneClassifier(nn.Module):
    def __init__(self, model=model_checkpoint, classes=2, head_dropout=0.2):
        super().__init__()

        self.model = RobertaModel.from_pretrained(model)
        hidden_size = self.model.config.hidden_size

        self.project = torch.nn.Sequential(
            torch.nn.Dropout(head_dropout),
            torch.nn.Linear(hidden_size, hidden_size),
            torch.nn.Dropout(head_dropout),
            torch.nn.Linear(hidden_size, classes)
        )

    def forward(self, input_ids, attention_mask=None):
        res = self.model.forward(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        res = res[0]
        res = res[:,0,:] # encoding for <s> token
        res = self.project(res)
        return res

    def parameters_num(self):
        return sum(p.numel() for p in self.parameters())

In [ ]:
def train(model,
          train_dataloader,
          valid_dataloader,
          steps,
          optimizer,
          accelerator,
          blind_steps=None,
          loss_fn=torch.nn.BCELoss(),
          main_metric=('f1', f1_score),
          additional_metrics=[],
          filepath='model_best_RoBERTa.pt',
          load_best=True,
          scheduler=None,
          losses_dict=None):

    if blind_steps == None:
        blind_steps = len(train_dataloader) // 4

    def evaluate():  # the first score returned is the main
        model.eval()

        y_trues = []
        y_hats = []

        loss = 0
        k = 0

        with torch.no_grad():
            for batch in valid_dataloader:

                (ids, mask), y_true = batch
                ids, mask = accelerator.prepare(ids), accelerator.prepare(mask)
                y_true = accelerator.prepare(y_true)
                hots = torch.nn.functional.one_hot(y_true, 2).to(dtype=torch.float)
                y_hat = torch.softmax(model.forward(input_ids=ids, attention_mask=mask),dim=-1)

                loss += float(loss_fn(y_hat, hots))
                k += 1

                for i in range(y_true.shape[0]):
                    y_trues.append(int(y_true[i]))
                    y_hats.append(1 if y_hat[i][0] < y_hat[i][1] else 0)

        scores = [(main_metric[0], main_metric[1](y_trues, y_hats))]

        for metric in additional_metrics:
            scores.append((metric[0], metric[1](y_trues, y_hats)))

        model.train()
        return scores + [('valid_loss', loss/k)]


    def render_scores(scores, step, best=None):
        print('{:05d} steps'.format(step), end=' ')

        for score in scores:
            print("| {}: {:.3f}".format(*score), end=' ')

        if best != None:
            print('| best_score: {:.3f}'.format(best))


    # initial scores
    scores = evaluate()
    render_scores(scores, 0)
    best_score = scores[0][1]
    torch.save(accelerator.unwrap_model(model).state_dict(), filepath)

    # logs
    if losses_dict != None:
        losses_dict['train_loss'] = []
        losses_dict['valid_loss'] = []
        losses_dict[main_metric[0]] = []

    epoch_loss = 0
    k = 0

    train_iter = iter(train_dataloader)
    model.train()

    for step in tqdm(range(steps)):

        # retrieving a batch
        try:
            batch = next(train_iter)
        except:
            train_iter = iter(train_dataloader)
            batch = next(train_iter)

        (ids, mask), y_true = batch
        ids, mask = accelerator.prepare(ids), accelerator.prepare(mask)
        y_true = accelerator.prepare(y_true)

        # prediction
        y_hat = torch.softmax(model.forward(input_ids=ids, attention_mask=mask),dim=-1)
        hots = torch.nn.functional.one_hot(y_true, 2).to(dtype=torch.float)
        loss = loss_fn(y_hat, hots)

        # backprop
        optimizer.zero_grad()
        accelerator.backward(loss)
        optimizer.step()

        if scheduler != None:
            scheduler.step()

        epoch_loss += float(loss)
        k += 1

        # evaluation
        if (step + 1) % blind_steps == 0:
            scores = evaluate() + [('train_loss', epoch_loss/k)]

            if losses_dict != None:
                losses_dict['valid_loss'].append(float(scores[-2][1]))
                losses_dict['train_loss'].append(float(scores[-1][1]))
                losses_dict[main_metric[0]].append(float(scores[0][1]))

            if scores[0][1] > best_score:
                best_score = scores[0][1]
                torch.save(accelerator.unwrap_model(model).state_dict(), filepath)

            render_scores(scores, step + 1, best=best_score)
            epoch_loss = 0
            k = 0

    if load_best:
        state_dict = torch.load(filepath)

        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            if "module." not in k:
                name = 'module.' + k
                new_state_dict[name] = v

        model.load_state_dict(new_state_dict)

In [ ]:
device = accelerator.device
model_checkpoint = "roberta-large"

tokenizer = RobertaTokenizer.from_pretrained(model_checkpoint)
model = RobertaModel.from_pretrained(model_checkpoint)

# Define the batch size and the number of folds
batch_size = 16
n_folds = 5
skf = StratifiedKFold(n_splits=n_folds)

# Iterating over each fold
for fold, (train_index, valid_index) in enumerate(skf.split(filtered_df, filtered_df['target'])):

    print(f'FOLD {fold + 1}')

    # Split the data into train and validation datasets for the current fold
    train_dataset = MyDataset(filtered_df.iloc[train_index], 'final_text', tokenizer)
    valid_dataset = MyDataset(filtered_df.iloc[valid_index], 'final_text', tokenizer)

    # Create DataLoaders for the current fold
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model for the current fold
    model = FinetuneClassifier(head_dropout=.1)
    model = nn.DataParallel(model)
    model.to(device)

    # Initialize optimizer for the current fold
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-6, weight_decay=1e-3)
    scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, total_iters=500)
    logs_dict = {}

    # Train the model for the current fold
    train(
      model,
      train_dataloader,
      valid_dataloader,
      1500,
      optimizer,
      accelerator,
      blind_steps=100,
      additional_metrics=[('precision', precision_score), ('recall', recall_score),('accuracy', accuracy_score)],
      filepath=f'model_best_RoBERTa_fold_{fold}.pt',
      losses_dict=logs_dict,
      scheduler=scheduler
    )

dataset_test = MyDatasetTest(filtered_test, 'final_text', tokenizer)

for fold in range(n_folds):
    model_path = f'/kaggle/working/model_best_RoBERTa_fold_{fold}.pt'
    state_dict = torch.load(model_path)

    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        if 'module.' not in k:
            name = 'module.' + k
            new_state_dict[name] = v

    model.load_state_dict(new_state_dict)

    predictions_df = pd.DataFrame()
    for i, (ids, mask) in tqdm(dataset_test):
        ids, mask = accelerator.prepare(ids), accelerator.prepare(mask)
        pred = model(input_ids=ids[None], attention_mask=mask[None])[0]
        y_hat = 1 if pred[0] < pred[1] else 0
        r = [int(i), y_hat]
        predictions_df = pd.concat([predictions_df, pd.DataFrame(np.array(r)[None,:], columns=['id', 'target'])])

    # Add the fold's predictions to the combined predictions DataFrame
    predictions_df.columns = ['id', f'target_fold_{fold}']
    if fold == 0:
        combined_predictions = predictions_df
    else:
        combined_predictions = combined_predictions.merge(predictions_df, on='id')

combined_predictions['target'] = combined_predictions.iloc[:, 1:].mean(axis=1).round().astype(int)
combined_predictions[['id', 'target']].to_csv('submission.csv', index=False)

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/tmp/ipykernel_31/3457697087.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['clean_text'] = filtered_df['text'].apply(clean_text)
/tmp/ipykernel_31/3457697087.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['combined'] = filter

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


FOLD 1


100%|████████████████████████████| 1497/1497 [00:01<00:00, 787.64it/s]
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

 93%|█████████▎| 1400/1500 [48:35<27:36, 16.56s/it]

01400 steps | f1: 0.814 | precision: 0.821 | recall: 0.807 | accuracy: 0.843 | valid_loss: 0.406 | train_loss: 0.306 | best_score: 0.830


100%|██████████| 1500/1500 [51:59<00:00,  2.08s/it]

01500 steps | f1: 0.805 | precision: 0.747 | recall: 0.873 | accuracy: 0.820 | valid_loss: 0.504 | train_loss: 0.294 | best_score: 0.830



100%|██████████| 3263/3263 [04:31<00:00, 12.03it/s]
